In [ ]:
import pandas as pd
import numpy as np
import datetime
from math import radians, cos, sin, asin, sqrt

# the following features will be in the resort specific dataset:
#    - dummy for every resort
#    - number of pictures
#    - number of followers
#    - dummy for every state
#    - PAF (Pure Awesomeness Factor from ZRankings)
#    - ranking within the state based on PAF
#    - population aggregated by state and by 3-, 4- and 5-digit zip codes and split by gender and age groups
#    - general weather statistics (precipitation, minimum temperature and maximum temperature, snowfall
#      and snow score preservation (preservation of snowpack due to exposure, altitude, latitude and rain incidence))

# stats_table contains the following features:
#    - name of resort -> convert into dummies with 'get_dummies'
#    - number of pictures
#    - number of followers
#    - state -> convert into dummies with 'get_dummies'
#    - zip code in order to merge the population data
#    - latitude and longitude in order to calculate distance to weather station

account_01 = pd.read_csv('results/stats_table.csv', sep = ',', encoding = "ISO-8859-1")
account_01['zip_code_5digits'] = account_01['zip_code'].apply(lambda x: str(x).zfill(5))
account_01['zip_code_4digits'] = account_01['zip_code_5digits'].apply(lambda x: x[0:4])
account_01['zip_code_3digits'] = account_01['zip_code_5digits'].apply(lambda x: x[0:3])

# import minimum and maximum elevation of every resort in order to calculate
# the difference in elevation with each weather station

elevation = pd.read_csv('other_data/resorts_elevation.csv', sep = ',', encoding = "ISO-8859-1")

# Silverton is not present in the ZRankings and will be therefore be excluded from the analysis
elevation = elevation[elevation['resort'] != 'Silverton']
elevation = elevation[['resort', 'min elevation (m)', 'max elevation (m)']]
elevation.rename(columns = {'min elevation (m)': 'minimum_elevation',
                            'max elevation (m)': 'maximum_elevation'}, inplace = True)

# resorts with few pictures or without an Instagram account will be excluded from the analysis
# Aspen Highlands, Aspen Mountain and Aspen Buttermilk have one Instagram account which will be merged to one ranking
# Squaw Valley and Alpine Meadows have one Instagram account which will be merged to one ranking
ZRankings = pd.read_csv('other_data/ZRankings.csv', sep = ',', encoding = "ISO-8859-1")
ZRankings = ZRankings[(ZRankings['country'] == 'United States') & 
                      (ZRankings['remarks'].isin([np.nan,
                                                  'average of Aspen Mountain and Aspen Highlands',
                                                  'average of Squaw Valley and Alpine Meadows']))]

ZRankings = ZRankings[['resort', 'PAF', 'overall', 'region', 'state', 'average yearly snowfall (cm)', 'total snow score preservation']]
ZRankings.rename(columns = {'overall': 'ranking_overall',
                            'region': 'ranking_region',
                            'state': 'ranking_state',
                            'average yearly snowfall (cm)': 'average_yearly_snowfall',
                            'total snow score preservation': 'tss_preservation'}, inplace = True)

print('The number of resorts in account is:', account_01.resort.count())
print('The number of resorts in elevation is:', elevation.resort.count())
print('The number of resorts in account is:', ZRankings.resort.count())

In [ ]:
# merge account data with elevation and ZRankings data

account_02 = account_01.merge(elevation, on = 'resort', how = 'inner')
account_03 = account_02.merge(ZRankings, on = 'resort', how = 'inner')

print('The number of resorts in account_01 is:', account_01.resort.count())
print('The number of resorts in account_02 is:', account_02.resort.count())
print('The number of resorts in account_03 is:', account_03.resort.count())

In [ ]:
# import the population data

zip_code_state = pd.read_csv('other_data/us_postal_codes.csv', sep = ',')
zip_code_state = zip_code_state[['Zip Code', 'State']]
zip_code_state.rename(columns = {'Zip Code': 'zip_code', 'State': 'state'}, inplace = True)
zip_code_state.head()

# 59% of people between 18 and 29 use Instagram, 33% of 30-49 year olds use Instagram
# Instagram is more popular by women (38% of online women use Instagram versus 28% of online men)
# source: https://sproutsocial.com/insights/new-social-media-demographics/#instagram...

population_zip_code = pd.read_csv('other_data/population_by_zip_2010.csv', sep = ',')

# population between 18 and 29 split by gender and zipcode

population_zip_code_18_29 = population_zip_code[population_zip_code['minimum_age'].isin([18, 20, 21, 22, 25])] \
            .groupby(['zipcode', 'gender']).population.sum() \
            .to_frame().reset_index()

population_zip_code_18_29 = population_zip_code_18_29.pivot_table(index = 'zipcode',
                                                                  columns = 'gender',
                                                                  values = 'population') \
            .reset_index() \
            .rename_axis(None, axis = 1)

population_zip_code_18_29['total'] = population_zip_code_18_29['female'] + population_zip_code_18_29['male']
population_zip_code_18_29.rename(columns = {'zipcode': 'zip_code',
                                            'female': 'female_18_29',
                                            'male': 'male_18_29',
                                            'total': 'total_18_29'}, inplace = True)

# population between 30 and 49 split by gender and zipcode

population_zip_code_30_49 = population_zip_code[population_zip_code['minimum_age'].isin([30, 35, 40, 45])] \
            .groupby(['zipcode', 'gender']).population.sum() \
            .to_frame().reset_index()

population_zip_code_30_49 = population_zip_code_30_49.pivot_table(index = 'zipcode',
                                                                  columns = 'gender',
                                                                  values = 'population') \
            .reset_index() \
            .rename_axis(None, axis = 1)

population_zip_code_30_49['total'] = population_zip_code_30_49['female'] + population_zip_code_30_49['male']
population_zip_code_30_49.rename(columns = {'zipcode': 'zip_code',
                                            'female': 'female_30_49',
                                            'male': 'male_30_49',
                                            'total': 'total_30_49'}, inplace = True)

# total population split by gender and zipcode

population_zip_code_total = population_zip_code[(population_zip_code['minimum_age'].isnull()) & 
                    (population_zip_code['maximum_age'].isnull())] \
            .groupby(['zipcode', 'gender']).population.sum() \
            .to_frame().reset_index()

population_zip_code_total = population_zip_code_total.pivot_table(index = 'zipcode',
                                                                  columns = 'gender',
                                                                  values = 'population') \
            .reset_index() \
            .rename_axis(None, axis = 1)
        
population_zip_code_total['total'] = population_zip_code_total['female'] + population_zip_code_total['male']
population_zip_code_total.rename(columns = {'zipcode': 'zip_code',
                                            'female': 'female_total',
                                            'male': 'male_total',
                                            'total': 'total_total'}, inplace = True)

# merging populations
populations = population_zip_code_18_29.merge(population_zip_code_30_49, on = 'zip_code', how = 'inner').merge(population_zip_code_total, on = 'zip_code', how = 'inner')

print('The number of records in 18-29 is:', population_zip_code_18_29.zip_code.count())
print('The number of records in 30-49 is:', population_zip_code_30_49.zip_code.count())
print('The number of records in total is:', population_zip_code_total.zip_code.count())
print('The number of records after the first merge:', populations.zip_code.count())

# merging the state to the zip code
populations = populations.merge(zip_code_state, on = 'zip_code', how = 'left')

print('The number of records after the second merge:', populations.zip_code.count())

# populations by state
populations_state = populations.groupby('state').sum().reset_index()
populations_state.drop('zip_code', axis = 1, inplace = True)
populations_state.rename(columns = {'female_18_29': 'female_18_29_state',
                                    'male_18_29':   'male_18_29_state',
                                    'total_18_29':  'total_18_29_state',
                                    'female_30_49': 'female_30_49_state',
                                    'male_30_49':   'male_30_49_state',
                                    'total_30_49':  'total_30_49_state',
                                    'female_total': 'female_total_state',
                                    'male_total':   'male_total_state',
                                    'total_total':  'total_total_state'}, inplace = True)

# change New Mexico to New_Mexico in order to merge
populations_state.replace(to_replace = 'New Mexico', value = 'New_Mexico', inplace = True)

# populations per zip code based on 3 -, 4 - and 5 digits
populations['zip_code_5digits'] = populations['zip_code'].apply(lambda x: str(x).zfill(5))
populations['zip_code_4digits'] = populations['zip_code_5digits'].apply(lambda x: x[0:4])
populations['zip_code_3digits'] = populations['zip_code_5digits'].apply(lambda x: x[0:3])

populations_3_digits = populations.groupby('zip_code_3digits').sum()
populations_3_digits.drop('zip_code', axis = 1, inplace = True)
populations_3_digits.columns = [str(col) + '_3_digits' for col in populations_3_digits.columns]
populations_3_digits.reset_index(inplace = True)

populations_4_digits = populations.groupby('zip_code_4digits').sum()
populations_4_digits.drop('zip_code', axis = 1, inplace = True)
populations_4_digits.columns = [str(col) + '_4_digits' for col in populations_4_digits.columns]
populations_4_digits.reset_index(inplace = True)

populations_5_digits = populations.groupby('zip_code_5digits').sum()
populations_5_digits.drop('zip_code', axis = 1, inplace = True)
populations_5_digits.columns = [str(col) + '_5_digits' for col in populations_5_digits.columns]
populations_5_digits.reset_index(inplace = True)

In [ ]:
# merge account data with population data

account_04 = account_03.merge(populations_5_digits, on = 'zip_code_5digits', how = 'inner')
account_05 = account_04.merge(populations_4_digits, on = 'zip_code_4digits', how = 'inner')
account_06 = account_05.merge(populations_3_digits, on = 'zip_code_3digits', how = 'inner')
account_07 = account_06.merge(populations_state, on = 'state', how = 'inner')

print('The number of resorts in account_03 is:', account_03.resort.count())
print('The number of resorts in account_04 is:', account_04.resort.count())
print('The number of resorts in account_05 is:', account_05.resort.count())
print('The number of resorts in account_06 is:', account_06.resort.count())
print('The number of resorts in account_07 is:', account_07.resort.count())

In [ ]:
# import the weather data

# create empty dataframes to fill up later
weather_total = pd.DataFrame()
account_stats_total = pd.DataFrame()

for i in range(0, len(account_02.values.tolist())):

    # PRCP = Precipitation (inches)
    # TMAX = Maximum temperature (degrees Fahrenheit)
    # TMIN = Minimum temperature (degrees Fahrenheit)

    weather_data = pd.read_csv('weather_data/weather_' + str(account_02.values.tolist()[i][2]) + '.csv', parse_dates = [5], dayfirst = True, low_memory = False)
    weather_data['latitude_resort'] = account_02.values.tolist()[i][4]
    weather_data['longitude_resort'] = account_02.values.tolist()[i][3]
    weather_data['min_elevation'] = account_02.values.tolist()[i][13]
    weather_data['max_elevation'] = account_02.values.tolist()[i][14]
    
    # there are some 'weird' values -> remove this low quality data (Alaska has some temperatures of -148 degrees F ...)
    weather_data = weather_data[weather_data['TMAX'] > -147]
    weather_data = weather_data[weather_data['TMIN'] > -147]
    
    # remove duplicates
    weather_data.drop_duplicates(keep = 'first', inplace = True)
    
    print('Resort:', account_02.values.tolist()[i][1])
    
    # calculate the distance from the resort to every weather station in the file
    
    def haversine(lon1, lat1, lon2, lat2):

        # convert decimal degrees to radians
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        
        # haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
        c = 2 * asin(sqrt(a))
        r = 6371 # radius of earth in kilometers (use 3956 for miles)
        
        return c * r
    
    weather_data['DISTANCE'] = weather_data.apply(lambda row: haversine(lon1 = row['LONGITUDE'],
                                                                        lat1 = row['LATITUDE'],
                                                                        lon2 = row['longitude_resort'],
                                                                        lat2 = row['latitude_resort']), axis = 1)
    
    # calculate the difference in elevation between the weather station and mid-mountain of the resort,
    # a positive number indicates a higher elevation of the weather station
    weather_data['ELEVATION_DIFFERENCE'] = (weather_data['ELEVATION'] - ((weather_data['max_elevation'] + weather_data['min_elevation']) / 2)) / 1000

    weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])
    
    # select only data between Jan 01, 2011 and Dec 31, 2016 so all statistics are based on the same data and have similar quality
    weather_data = weather_data[(weather_data['DATE'] >= datetime.date(2011, 1, 1)) & (weather_data['DATE'] <= datetime.date(2016, 12, 31))]
    
    weather_data = weather_data[['STATION', 'DATE', 'PRCP', 'TMAX', 'TMIN', 'DISTANCE', 'ELEVATION_DIFFERENCE']]
    weather_data.columns = ['station', 'date', 'precipitation_inches', 'temp_max', 'temp_min', 'distance', 'elevation_difference']
    
    # convert precipitation to mm and temperatures to degrees Celsius
    weather_data['precipitation_mm'] = weather_data['precipitation_inches'] * 2.54 * 10
    weather_data['temp_max_celsius'] = (weather_data['temp_max'] - 32) * 5 / 9
    weather_data['temp_min_celsius'] = (weather_data['temp_min'] - 32) * 5 / 9

    weather_stats = ['precipitation_mm', 'temp_max_celsius', 'temp_min_celsius']
    
    # create empty dataframes to fill up later
    weather_resort = pd.DataFrame()
    account_stats_resort = pd.DataFrame()
    
    for weather_stat in weather_stats:
                        
        # select nearest available observation for the weather statistic
        min_per_day = weather_data[weather_data[weather_stat].notnull()].groupby('date')['distance'].min().to_frame().reset_index()
        weather_data_optimum = weather_data.merge(min_per_day,
                                                  left_on = ['date', 'distance'],
                                                  right_on = ['date', 'distance'],
                                                  how = 'inner')
        
        weather_data_optimum = weather_data_optimum[['date', 'distance', 'elevation_difference', weather_stat]]
        print('Number of records in weather_data_optimum is:', weather_data_optimum.shape[0])
        
        weather_data_optimum.drop_duplicates(keep = 'first', inplace = True)
        print('Number of records in weather_data_optimum after removing possible duplicates is:', weather_data_optimum.shape[0])
        
        # correct the temperature for the difference in elevation (6 degrees celsius per 1000 meters)
        if (weather_stat == 'temp_max_celsius') | (weather_stat == 'temp_min_celsius'):
            weather_data_optimum[weather_stat] = weather_data_optimum[weather_stat] + weather_data_optimum['elevation_difference'] * 6
        
        # if the number of records does not equal 2192 (the number of days in 2011 - 2016) the program will stop
        if weather_data_optimum['date'].count() != 2192:
            print('Number of records for', weather_stat, 'near', account_02.values.tolist()[i][1], 'does not equal 2192.')
            exit()
            
        # save both minimum and maximum distance as well as minimum and maximum elevation difference per weather statistic 
        min_distance = weather_data_optimum['distance'].min()
        max_distance = weather_data_optimum['distance'].max()
        
        account_stats_temp = [account_01.values.tolist()[i][1],
                              weather_stat,
                              min_distance,
                              max_distance]
        
        account_stats_temp_df = pd.DataFrame(account_stats_temp).T
   
        # calculate the averages for every month of the year (1 - 12)
        weather_data_optimum['month'] = weather_data_optimum['date'].apply(lambda x: x.month)
        
        weather_temp = weather_data_optimum.pivot_table(index = None, 
                                                        columns = 'month', 
                                                        values = weather_stat, 
                                                        fill_value = 0, 
                                                        aggfunc = 'mean').reset_index()
        
        weather_temp.columns = [str(weather_stat) + '_' + str(col) for col in weather_temp.columns]
        weather_temp.drop([str(weather_stat) + '_index'], axis = 1, inplace = True)
        
        weather_resort = pd.concat([weather_resort, weather_temp], axis = 1)
        weather_resort['resort'] = account_02.values.tolist()[i][1]
        
        account_stats_resort = pd.concat([account_stats_resort, account_stats_temp_df], axis = 0)
        
    weather_total = pd.concat([weather_total, weather_resort], axis = 0)
    account_stats_total = pd.concat([account_stats_total, account_stats_resort], axis = 0)
        
    print('   OK!\n')

account_stats_total.columns = ['resort',
                               'weather statistic',
                               'minimum distance',
                               'maximum distance']

In [ ]:
# check the maximum distance per resort
account_stats_total.head()

In [ ]:
# save the statistics to a csv
account_stats_total.to_csv('results/account_statistics_resorts.csv', sep = ',', index = False)

In [ ]:
account_08 = account_07.merge(weather_total, on = 'resort', how = 'inner')

print('The number of resorts in account_07 is:', account_07.resort.count())
print('The number of resorts in account_08 is:', account_08.resort.count())

In [ ]:
account_09 = account_08.drop(['number_images_trash',
                              'zip_code_5digits',
                              'zip_code_4digits',
                              'zip_code_3digits'], axis = 1)

account_09['resort'] = account_09['resort'].astype('category')
account_09['state'] = account_09['state'].astype('category')

features_to_be_splitted = ['resort', 'state']

account_10 = pd.get_dummies(account_09, prefix = None, columns = features_to_be_splitted)

In [ ]:
account_10.columns

In [ ]:
# let's check a preview of the final dataframe
account_10.head()

In [ ]:
# how many missings does the final dataframe have?
account_10.isnull().sum().sum()

In [ ]:
# save the final dataframe to a csv
account_10.to_csv('results/resort_specific_dataset.csv', sep = ',', index = False)